In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import json
import pathlib
import pickle
import os
current_path = os.getcwd()

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('float_format', '{:f}'.format)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

from datetime import datetime, timedelta
import statistics
import time

import re

import pymysql  
from haversine import haversine

from geoband.API import *
import geopandas as gpd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster
import geoplot as gplt
import geoplot.crs as gcrs
import imageio
import mapclassify as mc

import random
from functools import reduce
from collections import defaultdict

from IPython.display import display
from tqdm.notebook import tqdm
from tqdm import tqdm, tqdm_notebook

import sklearn.cluster
import tensorflow as tf
import pydeck as pdk
import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')

import shapely
from shapely import wkt
from shapely.geometry import Polygon, Point, shape

# 최적화 solver
from mip import Model, xsum, maximize, BINARY  

# font 
import matplotlib.font_manager as font_manager
path = current_path+'/NanumBarunGothic.ttf'
fontprop = font_manager.FontProperties(fname=path)

font_dirs = [current_path, ]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
font_list = font_manager.createFontList(font_files)
font_manager.fontManager.ttflist.extend(font_list)
plt.rcParams["font.family"] = 'NanumGothic'
mpl.rcParams['font.family'] = 'NanumBarunGothic'

Using Python-MIP package version 1.5.3


In [2]:
# Data Load 
df_01 = pd.read_csv(current_path+'/input/1.수원시_버스정류장.csv')
df_02 = pd.read_csv(current_path+'/input/2.수원시_버스정류장별_승하차이력(1).csv')
df_03 = pd.read_csv(current_path+'/input/3.수원시_버스정류장별_승하차이력(2).csv')
df_04 = pd.read_csv(current_path+'/input/4.수원시_버스정류장별_승하차이력(3).csv')
df_05 = pd.read_csv(current_path+'/input/5.수원시_버스정류장별_승하차이력(4).csv')
df_06 = pd.read_csv(current_path+'/input/6.수원시_버스정류장별_승하차이력(5).csv')
df_07 = pd.read_csv(current_path+'/input/7.수원시_버스정류장별_노선현황.csv')
df_08 = pd.read_csv(current_path+'/input/8.수원시_지하철역_위치정보.csv')
df_09 = pd.read_csv(current_path+'/input/9.수원시_지하철역별_이용현황(2017~2019).csv')
df_10 = pd.read_csv(current_path+'/input/10.수원시_옥외광고물현황.csv')
df_11 = pd.read_csv(current_path+'/input/11.수원시_대기오염도_측정현황.csv')
df_12 = pd.read_csv(current_path+'/input/12.수원시_주차장현황.csv')
df_13 = pd.read_csv(current_path+'/input/13.수원시_기상데이터(2020).csv')
df_14 = pd.read_csv(current_path+'/input/14.수원시_시간대별_유동인구(2020).csv')
df_15 = pd.read_csv(current_path+'/input/15.수원시_성연령별_유동인구(2020).csv')
df_16 = pd.read_csv(current_path+'/input/16.수원시_요일별_유동인구(2020).csv')
df_17 = gpd.read_file(current_path+'/input/17.수원시_인구정보(고령)_격자.geojson')
df_18 = gpd.read_file(current_path+'/input/18.수원시_인구정보(생산가능)_격자.geojson')
df_19 = gpd.read_file(current_path+'/input/19.수원시_인구정보(유소년)_격자.geojson')
df_20 = gpd.read_file(current_path+'/input/20.수원시_교통노드.geojson')
df_21 = gpd.read_file(current_path+'/input/21.수원시_교통링크.geojson')
df_22 = gpd.read_file(current_path+'/input/22.수원시_상세도로망_LV6.geojson')
df_23 = pd.read_csv(current_path+'/input/23.수원시_평일_일별_시간대별_추정교통량_LV6.csv')
df_24 = pd.read_csv(current_path+'/input/24.수원시_평일_일별_혼잡빈도강도_LV6.csv')
df_25 = pd.read_csv(current_path+'/input/25.수원시_평일_일별_혼잡시간강도_LV6.csv')
df_26 = gpd.read_file(current_path+'/input/26.수원시_인도(2017).geojson')
df_27 = gpd.read_file(current_path+'/input/27.수원시_도로명주소(건물).geojson')
df_28 = gpd.read_file(current_path+'/input/28.수원시_건물연면적_격자.geojson')
df_29 = gpd.read_file(current_path+'/input/29.수원시_법정경계(시군구).geojson')
df_30 = gpd.read_file(current_path+'/input/30.수원시_법정경계(읍면동).geojson')
df_31 = gpd.read_file(current_path+'/input/31.수원시_행정경계(읍면동).geojson')
df_32 = gpd.read_file(current_path+'/input/32.수원시_지적도.geojson')

In [3]:
# pydeck function 
def line_string_to_coordinates(line_string): 
    if isinstance(line_string, shapely.geometry.linestring.LineString): 
        lon, lat = line_string.xy 
        return [[x, y] for x, y in zip(lon, lat)] 
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString): 
        ret = [] 
        for i in range(len(line_string)): 
            lon, lat = line_string[i].xy 
            for x, y in zip(lon, lat): 
                ret.append([x, y])
        return ret 

def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 
 
def multipolygon_to_center_coordinates(x): 
    lon, lat = x[0].centroid.xy 
    return [[x, y] for x, y in zip(lon, lat)]

def polygon_to_center_coordinates(x): 
    lon, lat = x.centroid.xy
    return [[x, y] for x, y in zip(lon, lat)] 

In [4]:
token = "pk.eyJ1IjoiZGx3b3FsczQzMjMiLCJhIjoiY2tscnR3bG95MDJwaDJ2bjUzcTBrc3h4cyJ9.WigDFX0Gm612haaz4zQ2hg"

In [5]:
bus = pd.read_excel(current_path+'/busdata/busdata_0318.xlsx')

# 성연령별 유동인구 

In [7]:
from bisect import bisect_left, bisect_right

In [6]:
# 일년치 평균 
pop_df = df_15.drop(['STD_YM'], axis=1).groupby(['lon', 'lat']).mean().reset_index()
print(pop_df.shape)
pop_df.head(3)

(38122, 14)


,lon,lat,MAN_FLOW_POP_CNT_10G,MAN_FLOW_POP_CNT_20G,MAN_FLOW_POP_CNT_30G,MAN_FLOW_POP_CNT_40G,MAN_FLOW_POP_CNT_50G,MAN_FLOW_POP_CNT_60GU,WMAN_FLOW_POP_CNT_10G,WMAN_FLOW_POP_CNT_20G,WMAN_FLOW_POP_CNT_30G,WMAN_FLOW_POP_CNT_40G,WMAN_FLOW_POP_CNT_50G,WMAN_FLOW_POP_CNT_60GU
0,126.927348,37.278703,0.000000,0.000000,0.000000,0.013333,0.026667,0.006667,0.000000,0.000000,0.000000,0.003333,0.003333,0.000000
1,126.927389,37.273295,0.020000,0.097000,0.221000,0.460000,0.689000,0.746000,0.026000,0.054000,0.098000,0.245000,0.367000,0.285000
2,126.927392,37.272844,0.035000,0.150000,0.335000,0.720000,0.997500,1.085000,0.042500,0.090000,0.150000,0.360000,0.540000,0.427500


In [31]:
bus['남자10대유동인구'], bus['남자20대유동인구'], bus['남자30대유동인구'], bus['남자40대유동인구'], bus['남자50대유동인구'], bus['남자60대유동인구'] = 0.1, 0.1, 0.1, 0.1, 0.1, 0.1 
bus['여자10대유동인구'], bus['여자20대유동인구'], bus['여자30대유동인구'], bus['여자40대유동인구'], bus['여자50대유동인구'], bus['여자60대유동인구'] = 0.1, 0.1, 0.1, 0.1, 0.1, 0.1 

In [10]:
pop_df = pop_df.sort_values(by = ['lon']).reset_index(drop = True)

In [32]:
def population(station, new):
    # 각 정류장에 대해 반복
    for i in tqdm(range(len(station))):
        
        # 계산량을 줄이기 위한 filter
        ## lon
        # sort_lon = new.sort_values(by = ['lon']).reset_index(drop = True)
        idx_lon = bisect_left(new['lon'], station['lon'][i])
        fltr_lon = new[(new['lon']<new.loc[idx_lon, 'lon']+0.001)&(new['lon']>new.loc[idx_lon, 'lon']-0.001)].reset_index(drop = True)
        ## lat
        sort_lat = fltr_lon.sort_values(by = ['lat']).reset_index(drop = True)
        idx_lat = bisect_left(sort_lat['lat'], station['lat'][i])
        fltr_lat = sort_lat[(sort_lat['lat']<sort_lat.loc[idx_lat, 'lat']+0.01)&(sort_lat['lat']>sort_lat.loc[idx_lat, 'lat']-0.01)].reset_index(drop = True)

        # filter된 데이터의 거리 계산
        distance = []
        for j in range(len(fltr_lat)):
            station_name = tuple(station[['lon','lat']].iloc[i])
            fltr_name = tuple(fltr_lat[['lon','lat']].iloc[j])
            distance.append(haversine(station_name, fltr_name, unit = 'km'))
            
        # 가장 가까운 격자의 유동인구
        station['남자10대유동인구'][i] = fltr_lat['MAN_FLOW_POP_CNT_10G'].iloc[np.argmin(distance)]
        station['남자20대유동인구'][i] = fltr_lat['MAN_FLOW_POP_CNT_20G'].iloc[np.argmin(distance)]
        station['남자30대유동인구'][i] = fltr_lat['MAN_FLOW_POP_CNT_30G'].iloc[np.argmin(distance)]
        station['남자40대유동인구'][i] = fltr_lat['MAN_FLOW_POP_CNT_40G'].iloc[np.argmin(distance)]
        station['남자50대유동인구'][i] = fltr_lat['MAN_FLOW_POP_CNT_50G'].iloc[np.argmin(distance)]
        station['남자60대유동인구'][i] = fltr_lat['MAN_FLOW_POP_CNT_60GU'].iloc[np.argmin(distance)]

        station['여자10대유동인구'][i] = fltr_lat['WMAN_FLOW_POP_CNT_10G'].iloc[np.argmin(distance)]
        station['여자20대유동인구'][i] = fltr_lat['WMAN_FLOW_POP_CNT_20G'].iloc[np.argmin(distance)]
        station['여자30대유동인구'][i] = fltr_lat['WMAN_FLOW_POP_CNT_30G'].iloc[np.argmin(distance)]
        station['여자40대유동인구'][i] = fltr_lat['WMAN_FLOW_POP_CNT_40G'].iloc[np.argmin(distance)]
        station['여자50대유동인구'][i] = fltr_lat['WMAN_FLOW_POP_CNT_50G'].iloc[np.argmin(distance)]
        station['여자60대유동인구'][i] = fltr_lat['WMAN_FLOW_POP_CNT_60GU'].iloc[np.argmin(distance)]        
        
    return station

In [33]:
bus = population(bus, pop_df)

100%|██████████| 516/516 [02:22<00:00,  3.62it/s]


In [34]:
bus.head(10)

,정류소ID,정류장명,쉘터,LED,LCD,LED+LCD복합형,알뜰형,lon,lat,중앙차로여부,운행노선수,정류장유형수,전체 승차 건수,초승 건수,환승 건수,전체 하차 건수,하차 건수,미태그 건수,주중배차간격(분),주말배차간격(분),주중상행배차횟수,주중하행배차횟수,주말상행배차횟수,주말하행배차횟수,초승_환승비율,초승_승차비율,초승_하차비율,환승_승차비율,환승_하차비율,승차_하차비율,전체승하차건수,승차비율,환승비율,하차비율,노선유형_경기순환버스(직행좌석형),노선유형_광역급행형시내버스,노선유형_맞춤형시내버스,노선유형_일반형시내버스,노선유형_좌석형시내버스,노선유형_직행좌석형시내버스,승차많은버스,환승많은버스,하차많은버스,평균상행운행시간,평균하행운행시간,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),오존(ppm),이산화질소(ppm),아황산가스(ppm),일산화탄소(ppm),500m내지하철역갯수,미세_봄,미세_여름,미세_가을,미세_겨울,초미세_봄,초미세_여름,초미세_가을,초미세_겨울,건축년도,전용면적(㎡),층,거래금액,면적당 금액,남자10대유동인구,남자20대유동인구,남자30대유동인구,남자40대유동인구,남자50대유동인구,남자60대유동인구,여자10대유동인구,여자20대유동인구,여자30대유동인구,여자40대유동인구,여자50대유동인구,여자60대유동인구
0,200000006,광교공원.경기대수원캠퍼스입구.연무시장,0,0,0,1,0,127.029464,37.300014,0,9,1,75.201863,69.878308,5.323555,33.747443,33.747443,0,98.312500,99.625000,61.909596,64.017341,52.366504,54.389856,12.771369,0.909413,14.400385,0.090587,1.121787,15.522172,108.949306,0.764578,0.075057,0.160365,0,0,0,8,0,0,7,0,0,0,0,49.458333,24.166667,0.018250,0.034458,0.003042,0.704167,0,61.000000,36.333333,39.666667,60.833333,28.333333,15.666667,17.166667,35.500000,2003,75,4,21700,288,2.365833,7.491667,9.765833,13.972500,20.946667,25.994167,1.820833,6.397500,6.496667,10.760000,17.704167,16.250833
1,200000008,문암골,0,0,0,1,0,127.027700,37.308946,0,1,1,77.498630,77.200000,0.298630,36.594521,36.594521,0,6.000000,6.000000,160.000000,175.000000,160.000000,175.000000,258.513761,0.996147,2.109605,0.003853,0.008161,2.117766,114.093151,0.676640,0.002617,0.320742,0,0,0,1,0,0,0,0,0,0,0,49.458333,24.166667,0.018250,0.034458,0.003042,0.704167,0,61.000000,36.333333,39.666667,60.833333,28.333333,15.666667,17.166667,35.500000,2003,75,4,21700,288,1.715833,5.220833,9.853333,15.887500,22.269167,30.544167,1.487500,4.343333,5.940833,10.950000,20.195000,19.827500
2,200000036,풍림아파트입구,1,0,0,1,0,126.995257,37.297929,0,1,2,73.186301,66.775342,6.410959,99.109589,99.109589,0,8.000000,10.500000,133.750000,133.750000,101.904762,101.904762,10.415812,0.912402,0.673753,0.087598,0.064686,0.738438,172.295890,0.387562,0.037209,0.575229,0,0,0,1,0,0,0,0,0,0,0,39.458333,23.125000,0.027917,0.024917,0.002958,0.562500,0,48.000000,29.833333,32.333333,47.666667,26.000000,16.333333,17.666667,32.500000,1989,84,1,29000,341,2.625000,4.275000,6.124167,8.574167,8.933333,7.345833,2.758333,3.272500,3.898333,7.241667,6.602500,4.957500
3,200000047,조원뉴타운,1,1,0,0,0,127.018224,37.304523,0,6,1,19.003357,18.755001,0.248356,19.983718,19.983718,0,132.583333,134.833333,22.351920,22.856804,15.679911,16.048959,36.497205,0.946993,0.431286,0.053007,0.011793,0.443079,38.987075,0.245771,0.008576,0.745654,0,0,0,6,0,0,0,0,4,0,0,39.458333,23.125000,0.027917,0.024917,0.002958,0.562500,0,48.000000,29.833333,32.333333,47.666667,26.000000,16.333333,17.666667,32.500000,2001,84,7,39450,467,4.225833,4.836667,6.171667,10.119167,11.035833,9.412500,4.180000,4.864167,5.212500,8.629167,8.676667,6.777500
4,200000066,수성중사거리,1,0,0,1,0,127.009723,37.292948,0,10,1,35.061027,25.297235,9.763791,35.543987,35.543987,0,90.777778,93.222222,49.311375,49.453025,37.186327,37.276274,2.312473,0.680678,1.278720,0.319322,0.591103,1.869823,70.605013,0.404698,0.189570,0.405732,0,0,0,9,0,0,0,0,0,0,0,39.458333,23.125000,0.027917,0.024917,0.002958,0.562500,0,48.000000,29.833333,32.333333,47.666667,26.000000,16.333333,17.666667,32.500000,2020,57,5,28750,496,2.580833,8.575000,13.912500,17.730000,20.731667,19.568333,2.568333,7.763333,8.851667,12.262500,13.874167,12.340000
5,200000068,율전중학교,1,1,0,0,0,126.965859,37.301088,0,1,1,0.506849,0.487671,0.019178,37.336986,37.336986,0,17.500000,20.000000,60.000000,60.000000,52.500000,52.500000,25.428571,0.962162,0.013061,0.037838,0.000514,0.013575,37.843836,0.012886,0.000507,0.986607,0,0,0,0,0,1,0,0,1,0,0,42.291667,22.250000,0.027917,0.025750,0.002792,0.454167,0,51.166667,32.333333,34.666667,51.000000,25.833333,15.500000,16.166667,31.500000,1997,59,19,24900,415,1.979167,7.279167,11.543333,15.732500,16.024167,13.685000,2.149167,6.416667,6.354167,7.405833,7.580000,7.642500
6,200000070,체육고교.신안아파트,1,1,0,0,0,126.

In [ ]:
bus.to_excel(current_path+'/busdata/bus_pop_0319.xlsx', index = False, encoding = 'CP949')